<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/AI_agents/CrewAI_Multi_AI_Agent_Blog_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries

In [1]:
!pip install crewai crewai_tools openai langchain-openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s e

## Import Libraries and add OpenAI key

In [2]:
import sys  # Provides access to system-specific parameters and functions
import os  # Enables interaction with the operating system
from crewai import Agent, Task, Crew, Process  # Imports core classes for defining agents, tasks, workflows, and processes
from langchain_openai import ChatOpenAI  # Integration with OpenAI's language models via LangChain
from crewai_tools import WebsiteSearchTool  # A tool for performing website searches and extracting relevant data

In [3]:
from google.colab import userdata
OpenAI_API = userdata.get('OPENAI_API_KEY')  # Fetching the stored OpenAI API key from the Colab userdata storage
os.environ['OPENAI_API_KEY'] = OpenAI_API  # Setting the fetched API key as an environment variable for use in the application

## Initialize web search tool and Crew

In [4]:
web_tool = WebsiteSearchTool() # Initializing a WebsiteSearchTool instance for performing web searches and extracting relevant data

In [6]:
# Initialize a ChatOpenAI instance with a specific temperature and model
llm  = ChatOpenAI(temperature=0.1, model="gpt-4o-mini")

# verbose = True, enables detailed logging

# Function to generate a blog based on a given topic
def generate_blog(topic):
    # Define the researcher agent responsible for gathering information
    researcher = Agent(
        role = "Internet Research",
        goal = f"""Conduct in-depth research on '{topic}' using reliable and diverse sources.
                 Identify key concepts, trends, examples, and statistics related to {topic}.
                 Your output should be a comprehensive, well-structured research document.""",
        verbose = True,
        llm=llm,
        backstory ="""You are a highly skilled researcher specializing in extracting, analyzing,
                     and consolidating information from the internet into actionable insights."""
    )

    # Define the content generator agent for drafting the blog
    generator = Agent(
        role='Content Generator',
        goal=f"""Draft a highly engaging and informative blog post on '{topic}'.
                 Include relevant examples, tutorials, visuals, and real-world applications.
                 Add emojis to make the blog reader-friendly. Use markdown for structure.""",
        verbose=True,
        allow_delegation=True,
        llm=llm,
        backstory="""You are an expert blogger known for creating detailed and captivating posts.
                     Your blogs are tutorial-like, enriched with real-world examples, and easy to follow."""
    )

    # Define the technical reviewer agent to ensure accuracy
    technical_reviewer = Agent(
        role="Technical Reviewer",
        goal=f"""Review the blog draft for technical accuracy, completeness, and relevance.
                 Ensure all examples, tutorials, and explanations are correct and useful.""",
        verbose=True,
        llm=llm,
        backstory="""You are a technical reviewer with expertise in validating and improving the
                     technical content of blogs, tutorials, and guides."""
    )

    # Define the copy editor agent for polishing the content
    copy_editor = Agent(
        role="Copy Editor",
        goal=f"""Polish the blog for grammar, style, and readability.
                 Ensure the content flows well and aligns with blogging best practices.""",
        verbose=True,
        llm=llm,
        backstory="""You are a skilled copy editor specializing in enhancing the quality, tone, and readability
                     of written content while preserving its original intent."""
    )

    # Define the markdown formatter agent for final formatting
    markdown_formatter = Agent(
        role="Markdown Formatter",
        goal=f"""Convert the finalized blog content into a properly structured markdown document.
                 Ensure the markdown formatting is clean, and includes proper headings, code blocks, and lists.""",
        verbose=True,
        llm=llm,
        backstory="""You are an expert in markdown formatting and structure, ensuring the content is clean,
                     organized, and ready for publishing."""
    )

    ## Define all tasks
    task_search = Task(
        description=f"""Perform extensive research on '{topic}' and compile a detailed,
                        easy-to-read research document (10,000 words).
                        Include examples, case studies, and all necessary data.""",
        expected_output=f"A well-organized research document about {topic}",
        max_inter=2,
        tools=[web_tool],
        agent=researcher
    )

    task_draft_blog = Task(
        description=f"""Use the research document to create a blog post (maximum 2,000 words).
                        Ensure the blog includes tutorials, code examples, real-world applications, and emojis.""",
        expected_output=f"A detailed and engaging blog post on {topic} in markdown format",
        agent=generator
    )

    task_review = Task(
        description=f"""Review the blog for technical accuracy and suggest any necessary improvements.
                        Ensure all examples, explanations, and tutorials are correct.""",
        expected_output=f"Feedback and revised content for technical correctness on {topic}",
        agent=technical_reviewer
    )

    task_edit = Task(
        description=f"""Edit the blog for grammar, style, and readability.
                        Ensure the content is polished and easy to read.""",
        expected_output=f"Polished and reader-friendly blog content for {topic}",
        agent=copy_editor
    )

    task_format_markdown = Task(
        description=f"""Format the finalized blog into a clean markdown .md document.
                        Ensure proper headings, lists, code blocks, and links are included.""",
        expected_output=f"A well-structured markdown file ready for publishing on {topic}",
        agent=markdown_formatter
    )

    # Define a crew to manage the agents and tasks
    crew = Crew(
        agents=[researcher, generator, technical_reviewer, copy_editor, markdown_formatter],
        tasks=[task_search, task_draft_blog, task_review, task_edit, task_format_markdown],
        verbose=True,
        process=Process.sequential)

    # Execute the process and return the result
    result = crew.kickoff()
    return result

## Running the Crew

Note: All the red, green colored text during the generation of blog is the model running the processes internally. If you want to stop seeing it, put verbose=False everywhere.

In [7]:
# Define the topic for the blog
topic = "Bagging vs Boosting in Machine Learning"
# Call the generate_blog function to create the blog on the given topic
result = generate_blog(topic)
# Print the final result of the blog generation process
print(result)

# Agent: Internet Research
## Task: Perform extensive research on 'Bagging vs Boosting in Machine Learning' and compile a detailed,
                        easy-to-read research document (10,000 words).
                        Include examples, case studies, and all necessary data.


Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]




# Agent: Internet Research
## Thought: I need to gather comprehensive information on Bagging and Boosting in Machine Learning, including definitions, key concepts, examples, case studies, and relevant statistics. I will start by searching for reliable sources that cover these topics in detail.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Bagging vs Boosting in Machine Learning\", \"website\": \"https://towardsdatascience.com\"}"
## Tool Output: 
Relevant Content:
Towards Data ScienceHomepageOpen in appSign inGet startedRidge Regression: A Robust Path to Reliable PredictionsRidge Regression: A Robust Path to Reliable PredictionsLearn how regularization reduces overfitting and improves model stability in linear regression.Niklas LangJan 30Thin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsThin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsA Practical Guide to Building Simple, Lightweight, and Task-Specific AI Services wit

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.15it/s]




# Agent: Internet Research
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Bagging vs Boosting in Machine Learning\", \"website\": \"https://www.analyticsvidhya.com\"}"
## Tool Output: 
Relevant Content:
Towards Data ScienceHomepageOpen in appSign inGet startedRidge Regression: A Robust Path to Reliable PredictionsRidge Regression: A Robust Path to Reliable PredictionsLearn how regularization reduces overfitting and improves model stability in linear regression.Niklas LangJan 30Thin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsThin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsA Practical Guide to Building Simple, Lightweight, and Task-Specific AI Services with Rust, Unsloth, and Llama.cppAlon AgmonJan 30Bite-Size Data Science: Falling for the Gambler’s FallacyBite-Size Data Science: Falling for the Gambler’s FallacyWhere the gambler’s fallacy shows up in data science and what to do about itJarom HuletJan 30Great Books fo

...



# Agent: Internet Research
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Bagging vs Boosting in Machine Learning\", \"website\": \"https://www.ibm.com\"}"
## Tool Output: 
Relevant Content:
Towards Data ScienceHomepageOpen in appSign inGet startedRidge Regression: A Robust Path to Reliable PredictionsRidge Regression: A Robust Path to Reliable PredictionsLearn how regularization reduces overfitting and improves model stability in linear regression.Niklas LangJan 30Thin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsThin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsA Practical Guide to Building Simple, Lightweight, and Task-Specific AI Services with Rust, Unsloth, and Llama.cppAlon AgmonJan 30Bite-Size Data Science: Falling for the Gambler’s FallacyBite-Size Data Science: Falling for the Gambler’s FallacyWhere the gambler’s fallacy shows up in data science and what to do about itJarom HuletJan 30Great Books for AI Enginee

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]




# Agent: Internet Research
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Bagging vs Boosting in Machine Learning\", \"website\": \"https://www.microsoft.com/en-us/research/\"}"
## Tool Output: 
Relevant Content:
Towards Data ScienceHomepageOpen in appSign inGet startedRidge Regression: A Robust Path to Reliable PredictionsRidge Regression: A Robust Path to Reliable PredictionsLearn how regularization reduces overfitting and improves model stability in linear regression.Niklas LangJan 30Thin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsThin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsA Practical Guide to Building Simple, Lightweight, and Task-Specific AI Services with Rust, Unsloth, and Llama.cppAlon AgmonJan 30Bite-Size Data Science: Falling for the Gambler’s FallacyBite-Size Data Science: Falling for the Gambler’s FallacyWhere the gambler’s fallacy shows up in data science and what to do about itJarom HuletJan 30Grea

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.56it/s]




# Agent: Internet Research
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Bagging vs Boosting in Machine Learning\", \"website\": \"https://www.educba.com\"}"
## Tool Output: 
Relevant Content:
Towards Data ScienceHomepageOpen in appSign inGet startedRidge Regression: A Robust Path to Reliable PredictionsRidge Regression: A Robust Path to Reliable PredictionsLearn how regularization reduces overfitting and improves model stability in linear regression.Niklas LangJan 30Thin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsThin Agents: Creating Lean AI Services with Local Fine-Tuned LLMsA Practical Guide to Building Simple, Lightweight, and Task-Specific AI Services with Rust, Unsloth, and Llama.cppAlon AgmonJan 30Bite-Size Data Science: Falling for the Gambler’s FallacyBite-Size Data Science: Falling for the Gambler’s FallacyWhere the gambler’s fallacy shows up in data science and what to do about itJarom HuletJan 30Great Books for AI Engi